# 使用Milvus和OpenAI入门
### 寻找你的下一本书

在这个笔记本中，我们将介绍如何使用OpenAI生成书籍描述的嵌入向量，并在Milvus中使用这些嵌入向量来找到相关的书籍。这个示例中的数据集来自HuggingFace datasets，包含了一百多万个标题-描述对。

让我们首先下载本笔记本所需的库：
- `openai` 用于与OpenAI嵌入服务进行通信
- `pymilvus` 用于与Milvus服务器进行通信
- `datasets` 用于下载数据集
- `tqdm` 用于显示进度条


In [1]:
! pip install openai pymilvus datasets tqdm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


安装所需的包后，我们就可以开始了。让我们从启动Milvus服务开始。要运行的文件是在此文件夹中找到的 `docker-compose.yaml`。这个命令会启动一个Milvus独立实例，我们将在本次测试中使用它。


In [7]:
! docker compose up -d


[+] Running 0/0
 ⠋ Network milvus  Creating                                                0.1s
[+] Running 1/1
 ⠿ Network milvus          Created                                         0.1s
 ⠋ Container milvus-minio  Creating                                        0.1s
 ⠋ Container milvus-etcd   Creating                                        0.1s
[+] Running 1/3
 ⠿ Network milvus          Created                                         0.1s
 ⠙ Container milvus-minio  Creating                                        0.2s
 ⠙ Container milvus-etcd   Creating                                        0.2s
[+] Running 1/3
 ⠿ Network milvus          Created                                         0.1s
 ⠹ Container milvus-minio  Creating                                        0.3s
 ⠹ Container milvus-etcd   Creating                                        0.3s
[+] Running 3/3
 ⠿ Network milvus               Created                                    0.1s
 ⠿ Container milvus-minio       Created 

在 Milvus 运行时，我们可以设置全局变量：
- HOST：Milvus 主机地址
- PORT：Milvus 端口号
- COLLECTION_NAME：Milvus 中集合的名称
- DIMENSION：嵌入的维度
- OPENAI_ENGINE：要使用的嵌入模型
- openai.api_key：您的 OpenAI 账户密钥
- INDEX_PARAM：用于集合的索引设置
- QUERY_PARAM：要使用的搜索参数
- BATCH_SIZE：一次要嵌入和插入多少个文本


In [27]:
import openai

HOST = 'localhost'
PORT = 19530
COLLECTION_NAME = 'book_search'
DIMENSION = 1536
OPENAI_ENGINE = 'text-embedding-3-small'
openai.api_key = 'sk-your_key'

INDEX_PARAM = {
    'metric_type':'L2',
    'index_type':"HNSW",
    'params':{'M': 8, 'efConstruction': 64}
}

QUERY_PARAM = {
    "metric_type": "L2",
    "params": {"ef": 64},
}

BATCH_SIZE = 1000


## Milvus
本部分涉及Milvus和为此用例设置数据库。在Milvus中，我们需要设置一个集合并对集合进行索引。


In [4]:
from pymilvus import connections, utility, FieldSchema, Collection, CollectionSchema, DataType

# 连接到 Milvus 数据库
connections.connect(host=HOST, port=PORT)


In [9]:
# 如果集合已存在，请将其移除。
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)


In [10]:
# 创建一个集合，包含id、标题和嵌入信息。
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='description', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)


In [11]:
# 在集合上创建索引并加载它。
collection.create_index(field_name="embedding", index_params=INDEX_PARAM)
collection.load()


## 数据集
在 Milvus 运行起来后，我们可以开始获取我们的数据了。Hugging Face Datasets 是一个包含许多不同用户数据集的中心，而在这个示例中，我们使用的是 Skelebor 的书籍数据集。该数据集包含超过100万本书的标题-描述对。我们将嵌入每个描述，并将其与标题一起存储在 Milvus 中。


In [12]:
import datasets

# 下载数据集并仅使用其中的 `train` 部分（文件大小约为800Mb）
dataset = datasets.load_dataset('Skelebor/book_titles_and_descriptions_en_clean', split='train')


/Users/filiphaltmayer/miniconda3/envs/haystack/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/Users/filiphaltmayer/.cache/huggingface/datasets/Skelebor___parquet/Skelebor--book_titles_and_descriptions_en_clean-3596935b1d8a7747/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


## 插入数据
现在我们已经将数据存储在我们的机器上，我们可以开始将其嵌入并插入到Milvus中。嵌入函数接受文本并以列表格式返回嵌入结果。


In [17]:
# 简单函数，用于将文本转换为嵌入表示。
def embed(texts):
    embeddings = openai.Embedding.create(
        input=texts,
        engine=OPENAI_ENGINE
    )
    return [x['embedding'] for x in embeddings['data']]



接下来的步骤是实际的插入操作。由于有这么多数据点，如果你想立即测试它，可以提前停止插入单元格并继续进行。这样做可能会降低结果的准确性，因为数据点较少，但仍然应该足够好。


In [18]:
from tqdm import tqdm

data = [
    [], # 标题
    [], # 描述
]

# 批量嵌入和插入
for i in tqdm(range(0, len(dataset))):
    data[0].append(dataset[i]['title'])
    data[1].append(dataset[i]['description'])
    if len(data[0]) % BATCH_SIZE == 0:
        data.append(embed(data[1]))
        collection.insert(data)
        data = [[],[]]

# 嵌入并插入余数 
if len(data[0]) != 0:
    data.append(embed(data[1]))
    collection.insert(data)
    data = [[],[]]



  0%|          | 1999/1032335 [00:06<57:22, 299.31it/s]  


KeyboardInterrupt: 

## 查询数据库
在我们的数据安全地插入Milvus后，现在可以执行查询操作了。查询接受一个字符串或字符串列表，并对它们进行搜索。结果会打印出您提供的描述以及包括结果分数、结果标题和结果书籍描述的结果。


In [31]:
import textwrap

def query(queries, top_k = 5):
    if type(queries) != list:
        queries = [queries]
    res = collection.search(embed(queries), anns_field='embedding', param=QUERY_PARAM, limit = top_k, output_fields=['title', 'description'])
    for i, hit in enumerate(res):
        print('Description:', queries[i])
        print('Results:')
        for ii, hits in enumerate(hit):
            print('\t' + 'Rank:', ii + 1, 'Score:', hits.score, 'Title:', hits.entity.get('title'))
            print(textwrap.fill(hits.entity.get('description'), 88))
            print()


In [32]:
query('Book about a k-9 from europe')


RPC error: [search], <MilvusException: (code=1, message=code: UnexpectedError, reason: code: CollectionNotExists, reason: can't find collection: book_search)>, <Time:{'RPC start': '2023-03-17 14:22:18.368461', 'RPC error': '2023-03-17 14:22:18.382086'}>


MilvusException: <MilvusException: (code=1, message=code: UnexpectedError, reason: code: CollectionNotExists, reason: can't find collection: book_search)>